In [1]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from transformers import BertTokenizer

In [2]:
import numpy as np

In [3]:
train_df = pd.read_csv('/content/New_News_Train.csv')
test_df = pd.read_csv('/content/New_News_Test.csv')

In [4]:
train_df

,Type,Removed_stopwords
0,1,اشتباك الحريري عون اتهامات لباسيل بالتمسك بالث...
1,1,عون الحريري اصبح غريب الاطوار
2,1,وزير الخارجية الامريكي ندرس سحب كامل قواتنا اف...
3,1,افغانستان استعدادات حثيثة لاجتماع تركيا وكابل ...
4,1,اندبندنت‮ ‬ ‬مفاوضات ‬سرية ‬‬كادت ‬تنقذ ‬القذافي
...,...,...
4995,1,اوروبا تبدا احصاء خسائر الفيضانات المدمرة
4996,1,قتل متظاهر بالرصاص خلال احتجاجات شح المياه منط...
4997,1,وسائل اعلام ايرانية تتحدث اندلاع احتجاجات الشوارع
4998,1,المفاوضات الافغانية تتواصل العاصمة القطرية الدوحة


In [5]:
tokenizer = BertTokenizer.from_pretrained('aubmindlab/bert-base-arabertv02')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [6]:
X_trainnews = train_df['Removed_stopwords']
y_trainnews = train_df['Type']

X_testnews = test_df['Removed_stopwords']
y_testnews = test_df['Type']

In [7]:
labels = train_df['Type'].values

In [8]:
import tensorflow as tf
from transformers import TFAutoModel, AutoTokenizer
from datasets import load_dataset

In [9]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [10]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py

In [11]:
!pip install tokenization

In [12]:
import tokenization
import tensorflow as tf
import tensorflow_hub as hub
from keras.utils import to_categorical
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [13]:
m_url = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/2'
bert_layer = hub.KerasLayer(m_url, trainable=True)

In [14]:
def bert_encode(texts, tokenizer, max_len=512):
    all_tokens = []
    all_masks = []
    all_segments = []

    for text in texts:
        text = tokenizer.tokenize(text)

        text = text[:max_len-2]
        input_sequence = ["[CLS]"] + text + ["[SEP]"]
        pad_len = max_len-len(input_sequence)

        tokens = tokenizer.convert_tokens_to_ids(input_sequence) + [0] * pad_len
        pad_masks = [1] * len(input_sequence) + [0] * pad_len
        segment_ids = [0] * max_len

        all_tokens.append(tokens)
        all_masks.append(pad_masks)
        all_segments.append(segment_ids)

    return np.array(all_tokens), np.array(all_masks), np.array(all_segments)

In [15]:
def build_model(bert_layer, max_len=512):
    input_word_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_word_ids")
    input_mask = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="input_mask")
    segment_ids = tf.keras.Input(shape=(max_len,), dtype=tf.int32, name="segment_ids")

    pooled_output, sequence_output = bert_layer([input_word_ids, input_mask, segment_ids])

    clf_output = sequence_output[:, 0, :]

    lay = tf.keras.layers.Dense(64, activation='relu')(clf_output)
    lay = tf.keras.layers.Dropout(0.2)(lay)
    lay = tf.keras.layers.Dense(32, activation='relu')(lay)
    lay = tf.keras.layers.Dropout(0.2)(lay)
    out = tf.keras.layers.Dense(4, activation='softmax')(lay)

    model = tf.keras.models.Model(inputs=[input_word_ids, input_mask, segment_ids], outputs=out)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

    return model

In [16]:
label = preprocessing.LabelEncoder()
y = label.fit_transform(train_df['Type'])
y = to_categorical(y)


In [17]:
from transformers import BertTokenizer

In [18]:
#tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [19]:
max_len = 250
train_input = bert_encode(train_df.Removed_stopwords.values, tokenizer, max_len=max_len)
test_input = bert_encode(test_df.Removed_stopwords.values, tokenizer, max_len=max_len)
train_labels = y

In [20]:
labels = label.classes_
print(labels)

[0 1 2 3]


In [21]:
model = build_model(bert_layer, max_len=max_len)

In [22]:
model.fit(train_input, train_labels, epochs=1, batch_size=35)


InvalidArgumentError: Graph execution error:

Detected at node word_embeddings/Gather defined at (most recent call last):
<stack traces unavailable>
indices[5501] = 43173 is not in [0, 30522)
	 [[{{node word_embeddings/Gather}}]] [Op:__inference_train_function_50781]

In [ ]:
!pip show transformers

In [ ]:
!pip install transformers==4.28.1